## Imports and API Key Setup

In [72]:
# Imports
import os
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langgraph.graph import Graph
import logging
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from typing import Dict, Any

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

## Load Dell Laptop Product Pages

In [73]:
urls = [
    'https://www.dell.com/en-us/shop/laptops-2-in-1-pcs/inspiron-14-2-in-1-laptop/spd/inspiron-14-7435-2-in-1-laptop',
    'https://www.dell.com/en-us/shop/laptops-2-in-1-pcs/latitude-5450-laptop/spd/latitude-14-5450-laptop',
    'https://www.dell.com/en-us/shop/laptops-2-in-1-pcs/latitude-7450-laptop-or-2-in-1/spd/latitude-14-7450-laptop',
    'https://www.dell.com/en-us/shop/laptops-2-in-1-pcs/xps-14-laptop/spd/xps-14-9440-laptop'
]
loader = WebBaseLoader(urls)
documents = loader.load()
print(f'Loaded {len(documents)} documents.')

Loaded 4 documents.


## Split Documents and Create Vector Store:

In [74]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(splits, embeddings)
retriever = vectorstore.as_retriever(
    search_type='mmr',  # Use Maximum Marginal Relevance for better diversity
    search_kwargs={
        'k': 6,  # Increase number of documents returned
        'fetch_k': 10,  # Fetch more documents for MMR selection
        'lambda_mult': 0.7  # Balance between relevance and diversity
    }
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


## Testing Specific Laptop Configuration Requests

In [75]:
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a helpful assistant that recommends Dell laptops based on user requirements. 
    Use the following context to make your recommendation. Always specify the exact model name (e.g., Dell Latitude 5450, XPS 14, etc.) 
    and configuration number if available. Include specific details about the model's features and why it matches the user's requirements.

    Context:
    {context}"""),
    ("human", "{question}")
])

def process_response(response: str, query: str) -> Dict[str, Any]:
    if not response or not isinstance(response, str):
        logger.error(f"Invalid response received: {response}")
        return {'response': "I apologize, but I couldn't generate a valid recommendation. Please try rephrasing your request.", 'status': 'error'}
    
    try:
        # Extract processor and storage from query
        processor = query.split('that has')[1].split('and')[0].strip() if 'that has' in query else 'Not specified'
        storage = query.split('has')[2].strip() if 'and has' in query else 'Not specified'
        
        formatted_response = f"""Based on your requirements, here is my recommendation:

{response}

Key Features:
- Model: [Model name from response]
- Processor: {processor}
- Storage: {storage}

Additional Considerations:
1. Please verify the exact specifications with Dell's website
2. Consider warranty and support options
3. Check for any current promotions or discounts

Would you like more specific information about any of these aspects?"""
        return {'response': formatted_response, 'status': 'success'}
    except Exception as e:
        logger.error(f"Error processing response: {str(e)}", exc_info=True)
        return {'response': f"An error occurred while processing your request: {str(e)}", 'status': 'error'}

# Create the chain with logging
def log_retriever_output(query: str) -> str:
    logger.info(f"Retriever received query: {query}")
    docs = retriever.invoke(query)
    logger.info(f"Retriever returned {len(docs)} documents")
    for i, doc in enumerate(docs):
        logger.info(f"Document {i+1} content: {doc.page_content[:200]}...")
    context = "\n".join([doc.page_content for doc in docs])
    logger.info(f"Combined context length: {len(context)} characters")
    return context

def log_prompt_output(inputs: dict) -> dict:
    logger.info("Formatting prompt with context and question")
    logger.info(f"Input to prompt formatting: {inputs}")
    messages = prompt.format_messages(**inputs)
    logger.info(f"Formatted messages: {messages}")
    return messages

def log_llm_output(messages: list) -> str:
    logger.info("Sending messages to LLM")
    logger.info(f"Messages being sent to LLM: {messages}")
    response = llm.invoke(messages)
    logger.info(f"LLM response type: {type(response)}")
    logger.info(f"Full LLM response: {response.content}")
    return response.content

# Create the chain with proper response handling
chain = (
    RunnablePassthrough.assign(
        context=lambda x: log_retriever_output(x["question"])
    )
    | log_prompt_output
    | log_llm_output
    | StrOutputParser()
)
logger.info("Created recommendation chain")

def recommend_laptop(state: Dict[str, Any]) -> Dict[str, Any]:
    query = state['query']
    logger.info(f"Processing recommendation request for query: {query}")
    
    try:
        logger.info("Invoking recommendation chain")
        logger.info(f"Input to chain: {{'question': {query}}}")
        response = chain.invoke({"question": query})
        logger.info(f"Chain response: {response}")
        
        if not response:
            logger.error("Empty response received from chain")
            return {'response': "I apologize, but I couldn't generate a valid recommendation. Please try rephrasing your request.", 'status': 'error'}
        
        result = process_response(response, query)
        logger.info(f"Processed result: {result}")
        return result  # Make sure this return value is being passed through the graph
    except Exception as e:
        logger.error(f"Error in recommendation process: {str(e)}", exc_info=True)
        return {'response': f"An error occurred while processing your request: {str(e)}", 'status': 'error'}

# Update the graph
graph = Graph()
graph.add_node('recommendation', recommend_laptop)
graph.set_entry_point('recommendation')
graph.set_finish_point('recommendation')  # Add this line to properly handle the return value
app = graph.compile()
logger.info("Graph compiled successfully")

# Test cases
def test_recommendation(query: str) -> None:
    logger.info(f"Testing with query: {query}")
    try:
        logger.info("Invoking graph with query")
        result = app.invoke({'query': query})
        logger.info(f"Raw result from graph: {result}")
        
        if result and isinstance(result, dict) and 'response' in result:
            print('Response:', result['response'])
        else:
            print("Error: Invalid response format received from the agent")
            logger.error(f"Invalid result format: {result}")
    except Exception as e:
        print(f"Error during recommendation: {str(e)}")
        logger.error(f"Error during recommendation: {str(e)}", exc_info=True)

# Test cases
print("\n=== Test Case 1 ===")
test_recommendation("I want a dell computer for travel that has Intel® Core™ 7 150U.")

print("\n=== Test Case 2 ===")
test_recommendation("I want a dell computer that has Intel® Core™ Ultra 5 135U vPro® and has 512 GB SSD.")

print("\n=== Test Case 3 ===")
test_recommendation("I want a dell computer that has Intel® Core™ Ultra 7 165U vPro® and 1TB SSD.")

print("\n=== Test Case 4 ===")
test_recommendation("I want a light weight XPS computer with Intel® Core™ Ultra 7 165U vPro® and 1 TB SSD.")




INFO:__main__:Created recommendation chain
INFO:__main__:Graph compiled successfully
INFO:__main__:Testing with query: I want a dell computer for travel that has Intel® Core™ 7 150U.
INFO:__main__:Invoking graph with query
INFO:__main__:Processing recommendation request for query: I want a dell computer for travel that has Intel® Core™ 7 150U.
INFO:__main__:Invoking recommendation chain
INFO:__main__:Input to chain: {'question': I want a dell computer for travel that has Intel® Core™ 7 150U.}
INFO:__main__:Retriever received query: I want a dell computer for travel that has Intel® Core™ 7 150U.



=== Test Case 1 ===


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:__main__:Retriever returned 6 documents
INFO:__main__:Document 1 content: access to Express Checkout Learn More about financing offers  |Order Code s0023l5450usvp  Bringing work closer to you14-inch AI laptop for intelligent, scalable performance. Featuring Intel® Core™ Ult...
INFO:__main__:Document 2 content: access to Express Checkout Learn More about financing offers  |Order Code s0023l5450usvp  Bringing work closer to you14-inch AI laptop for intelligent, scalable performance. Featuring Intel® Core™ Ult...
INFO:__main__:Document 3 content: access to Express Checkout Learn More about financing offers  |Order Code s0023l5450usvp  Bringing work closer to you14-inch AI laptop for intelligent, scalable performance. Featuring Intel® Core™ Ult...
INFO:__main__:Document 4 content: access to Express Checkout Learn More about financing offers  |Order Code s0023l5450usvp  Bringing work closer to you14-in

Response: Based on your requirements, here is my recommendation:

I recommend the Dell Latitude 5450 with Configuration 3. This model features the Intel® Core™ Ultra 7 165U processor, which is a powerful and efficient choice for travel. It also comes with 16 GB DDR5 memory, a 512 GB SSD for fast storage, and a 14" Touch FHD display, making it a versatile option for both work and entertainment on the go. With its intelligent, scalable performance and compact 14-inch size, the Latitude 5450 Configuration 3 is an excellent choice for your travel needs.

Key Features:
- Model: [Model name from response]
- Processor: Intel® Core™ 7 150U.
- Storage: Not specified

Additional Considerations:
1. Please verify the exact specifications with Dell's website
2. Consider warranty and support options
3. Check for any current promotions or discounts

Would you like more specific information about any of these aspects?

=== Test Case 2 ===


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:__main__:Retriever returned 6 documents
INFO:__main__:Document 1 content: access to Express Checkout Learn More about financing offers  |Order Code s0023l5450usvp  Bringing work closer to you14-inch AI laptop for intelligent, scalable performance. Featuring Intel® Core™ Ult...
INFO:__main__:Document 2 content: access to Express Checkout Learn More about financing offers  |Order Code s0023l5450usvp  Bringing work closer to you14-inch AI laptop for intelligent, scalable performance. Featuring Intel® Core™ Ult...
INFO:__main__:Document 3 content: access to Express Checkout Learn More about financing offers  |Order Code s0023l5450usvp  Bringing work closer to you14-inch AI laptop for intelligent, scalable performance. Featuring Intel® Core™ Ult...
INFO:__main__:Document 4 content: access to Express Checkout Learn More about financing offers  |Order Code s0023l5450usvp  Bringing work closer to you14-in

Response: Based on your requirements, here is my recommendation:

I recommend the **Dell Latitude 5450 Configuration 2** priced at $1,399.00. This laptop features an **Intel® Core™ Ultra 5 135U vPro®** processor and comes with a **512 GB SSD**, meeting your requirements perfectly. It also includes **16 GB DDR5 RAM**, **Windows 11 Pro**, and a **14" Non-Touch FHD** display. This configuration offers a balance of performance and storage capacity, making it a suitable choice for your needs.

Key Features:
- Model: [Model name from response]
- Processor: Intel® Core™ Ultra 5 135U vPro®
- Storage: 512 GB SSD.

Additional Considerations:
1. Please verify the exact specifications with Dell's website
2. Consider warranty and support options
3. Check for any current promotions or discounts

Would you like more specific information about any of these aspects?

=== Test Case 3 ===


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:__main__:Retriever returned 6 documents
INFO:__main__:Document 1 content: access to Express Checkout Learn More about financing offers  |Order Code s0023l5450usvp  Bringing work closer to you14-inch AI laptop for intelligent, scalable performance. Featuring Intel® Core™ Ult...
INFO:__main__:Document 2 content: access to Express Checkout Learn More about financing offers  |Order Code s0023l5450usvp  Bringing work closer to you14-inch AI laptop for intelligent, scalable performance. Featuring Intel® Core™ Ult...
INFO:__main__:Document 3 content: access to Express Checkout Learn More about financing offers  |Order Code s0023l5450usvp  Bringing work closer to you14-inch AI laptop for intelligent, scalable performance. Featuring Intel® Core™ Ult...
INFO:__main__:Document 4 content: access to Express Checkout Learn More about financing offers  |Order Code s0023l5450usvp  Bringing work closer to you14-in

Response: Based on your requirements, here is my recommendation:

Based on your requirements for an Intel® Core™ Ultra 7 165U vPro® processor and a 1TB SSD, I recommend the Dell Latitude 5450 with Configuration 3. This model features the Intel® Core™ Ultra 7 165U vPro® processor, 16 GB DDR5 RAM, a 512 GB SSD, and a 14" Touch FHD display. While it comes with a 512 GB SSD, you can easily upgrade the storage to 1TB to meet your needs. This laptop provides intelligent, scalable performance ideal for your requirements.

Key Features:
- Model: [Model name from response]
- Processor: Intel® Core™ Ultra 7 165U vPro®
- Storage: Not specified

Additional Considerations:
1. Please verify the exact specifications with Dell's website
2. Consider warranty and support options
3. Check for any current promotions or discounts

Would you like more specific information about any of these aspects?

=== Test Case 4 ===


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:__main__:Retriever returned 6 documents
INFO:__main__:Document 1 content: SpecsFeaturesReviewsSupportTech SpecsProcessorIntel® Core™ Ultra 7 155H (16 cores, up to 4.8 GHz)Operating System(Dell Technologies recommends Windows 11 Pro for business)Windows 11 Home, English, Fre...
INFO:__main__:Document 2 content: Price $1,499.99Intel® Core™ Ultra 7 155H, 16 coresWindows 11 HomeIntel® Arc™ Graphics16 GB LPDDR5X512 GB SSD14.5" Non-Touch FHD+ Tech Specs  View Special Offers  Select Selected Configuration 1 Config...
INFO:__main__:Document 3 content: Price $1,499.99Intel® Core™ Ultra 7 155H, 16 coresWindows 11 HomeIntel® Arc™ Graphics16 GB LPDDR5X512 GB SSD14.5" Non-Touch FHD+ Tech Specs  View Special Offers  Select Selected Configuration 1 Config...
INFO:__main__:Document 4 content: Price $1,499.99Intel® Core™ Ultra 7 155H, 16 coresWindows 11 HomeIntel® Arc™ Graphics16 GB LPDDR5X512 GB SSD14.5" Non-Touc

Response: Based on your requirements, here is my recommendation:

I recommend the Dell XPS 14 Laptop with the configuration option of Intel® Core™ Ultra 7 155H, Windows 11 Home, NVIDIA® GeForce RTX™ 4050, 32 GB LPDDR5X RAM, and a 1 TB SSD. This model offers powerful performance with the Intel® Core™ Ultra 7 155H processor and ample storage with the 1 TB SSD. Additionally, it features a dedicated graphics card for enhanced visual performance. The XPS 14 is lightweight and portable, making it a great choice for users looking for a high-performance yet portable laptop.

Key Features:
- Model: [Model name from response]
- Processor: Not specified
- Storage: Not specified

Additional Considerations:
1. Please verify the exact specifications with Dell's website
2. Consider warranty and support options
3. Check for any current promotions or discounts

Would you like more specific information about any of these aspects?
